<a href="https://colab.research.google.com/github/QeyCoder/WhatsApp-Chat-Analysis/blob/master/Chat_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

sample_data


In [0]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

**Uncomment the codes to download the file from google drive **

In [0]:
# auth.authenticate_user()
# gauth = GoogleAuth()

In [0]:
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [0]:
# fluff, id = link.split('=')

In [0]:
# import pandas as pd
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('Filename.csv')  


In [0]:
# df = pd.read_csv('Filename.csv', header=None,error_bad_lines=False)

FileNotFoundError: ignored

In [0]:
df3.info()

In [0]:
#df.tail()

In [0]:
#Data Clean

In [0]:
import regex as re
def checkForDateTime(inputMessage):
  pattern='^([0-2][0-9]|(3)[0-1])(\/)(((0)[0-9])|((1)[0-2]))(\/)(\d{2}|\d{4}),\s([0-9]{0,1}[0-9]):([0-9][0-9])'
  result = re.match(pattern,inputMessage)
  if result:
    return True
  return False 



In [0]:
def findAuthor(inputMessage):

  pattern=  patterns = [
        '([\w]+)\s*:',                        # First Name
        '([\w]+[\s]+[\w]+):',              # First Name + Last Name
        '([\w]+[\s]+[\w]+[\s]+[\w]+):',    # First Name + Middle Name + Last Name
        '([+]\d{2} \d{5} \d{5}):',         # Mobile Number (India)
        '([+]\d{2} \d{3} \d{3} \d{4}):',   # Mobile Number (US)
        '([+]\d{2} \d{4} \d{7})'           # Mobile Number (Europe)
    ]
  pattern = '^' + '|'.join(patterns)
  result = re.match(pattern, inputMessage)
  if result:
        return True
  return False

In [0]:
def findDataPoint(line):
  line= line.encode('ascii', 'ignore').decode('ascii')
  splitedLine = line.split(" - ")
  dateTime=splitedLine[0]
  date,time= dateTime.split(", ")
  message = ' '.join(splitedLine[1:])
  if findAuthor(message):
    splittedMessage = message.split(': ')
    author= splittedMessage[0]
    message=' '.join(splittedMessage[1:])
  else:
    author=None
  return date,time,author,message    
                


In [0]:
parsedData = [] 
conversationPath = 'Filename.csv' 
with open(conversationPath, encoding="utf-8") as fp:
    fp.readline()
        
    messageBuffer = [] 
    date, time, author = None, None, None 
    
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip() 
        if checkForDateTime(line): 
            if len(messageBuffer) > 0:
                parsedData.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear() 
            date, time, author, message = findDataPoint(line)
            messageBuffer.append(message) 
        else:
            messageBuffer.append(line) 
     

In [0]:
df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])

In [0]:
author_count =df.Author.value_counts()
top_10_author_val_count =author_count.head(10)
# top_10_author_val_count.plot.barh()


In [0]:
media_messages_df = df[df['Message'] == '<Media omitted>']

In [0]:
 clean_Media_df = df.drop(media_messages_df.index)

In [0]:
# clean_Media_df.tail()

In [0]:
cleaned_author =clean_Media_df.drop(df[df['Author'].isnull()].index)

In [0]:
supr_clean_msg = cleaned_author.drop(cleaned_author[cleaned_author['Message']=="Waiting for this message"].index)

In [0]:
supr_clean_msg['Letter_count']= supr_clean_msg['Message'].apply(lambda s: len(s))
supr_clean_msg['Word_count']= supr_clean_msg['Message'].apply(lambda s: len(s.split(' ')))

In [0]:
author_count =supr_clean_msg.Author.value_counts()
top_10_author_val_count =author_count.head(10)
# top_10_author_val_count.plot.barh()


In [0]:
supr_clean_msg[['Date','Author','Message','Time']].describe()

In [0]:
supr_clean_msg[['Letter_count','Word_count']].describe()

In [0]:
supr_clean_msg['Date'].value_counts().head(10).plot.barh() 

In [0]:
supr_clean_msg['Time'].value_counts().head(10).plot.barh() 

In [0]:
from collections import Counter
x = Counter(" ".join(supr_clean_msg["Message"]).lower().split()).most_common(200)


In [0]:
# pd.DataFrame(x).sample(20)

In [0]:
x